# Create plaquette using the RPNG format

## 1. RPNG format

```
-z1- -z2- -z3- -z4-
rpng rpng rpng rpng
```

(r) data qubit reset basis or h or -  
(p) data basis for the controlled operation (x means CNOT controlled on the ancilla and targeting the data qubit, y means CY, z means CZ)  
(n) time step (positive integers, all distinct, typically in 1-5)  
(g) data qubit measure basis or h or -

Assumptions on the circuit:
- if not otherwise stated, a basis can be {x,y,z}
- the ancilla is always initialized in $\ket{+}$ and measured in the X basis
- the ancilla is always the control qubit for the CNOT and CZ gates
- time step of r same as ancilla reset (default 0)
- time step of g same as ancilla measurement (default 6)

## 2. Extended RPNG format

```
z0z5 -xz1- -xz2- -xz3- -xz4-
pnpn rppng rppng rppng rppng
```

(p) ancilla init basis  
(n) time step  
(p) ancilla measure basis  
(n) time step

(r) data qubit reset basis or h or -  
(pp) ancilla-data 2-qubit bases (xz means CNOT targeting the ancilla)
(n) time step (positive integers, all distinct)
(g) data qubit measure basis or h or -

Assumptions on the circuit:
- if not otherwise stated, a basis can be {x,y,z}
- at least one of the (pp) must be z, indicating the control qubit
- time step of r same as ancilla reset
- time step of g same as ancilla measurement
- the time step of every (pp) must be in [ancilla init time + 1, ancilla measure time -1]  

## 3. Test the functionality

In [1]:
from pathlib import Path
import stim

from tqec import (
    Plaquette,
    PlaquetteQubits,
    QubitMap,
    ScheduledCircuit,
    SquarePlaquetteQubits,
    RPNG,
    RGN,
    RPNGDescription,
)

ASSETS_FOLDER = Path("../../assets/").resolve()

In [2]:
# -- Basic use case --

# Write the RPNG string
rpng_str = '-x5h -z2z -x3x hz1-'
qubits = SquarePlaquetteQubits()

# Create the corresponding RPNGDescription object
try:
    desc = RPNGDescription.from_string(rpng_str)
except ValueError as err:
    print(f'{err}')

# Create the corresponding Plaquette
qubits = SquarePlaquetteQubits()
plaq = desc.get_plaquette(qubits = qubits)

# Print the circuit associated with the plaquette
print(plaq.circuit.get_circuit(include_qubit_coords=True))

0: has meas -> ExtendedBasisEnum.H
1: has meas -> ExtendedBasisEnum.Z
2: has meas -> ExtendedBasisEnum.X
3: has reset -> ExtendedBasisEnum.H
QUBIT_COORDS(-1, -1) 0
QUBIT_COORDS(1, -1) 1
QUBIT_COORDS(-1, 1) 2
QUBIT_COORDS(1, 1) 3
QUBIT_COORDS(0, 0) 4
H 3
RX 4
TICK
CZ 4 3
TICK
CZ 4 1
TICK
CX 4 2
TICK
TICK
CX 4 0
TICK
H 0
M 1
MX 2 4


In [5]:
# Invalid plaquettes
rpng_invalide_examples = [
    '----- ---- ---- ----', # wrong length of values
    '---- ---- --- ----', # wrong length of values
    '-z1- -z2- ---- -z4-', # wrong number of 2Q gates
    '-z1- -z4- -z3- -z4-', # wrong times for the 2Q gates
    '-z1- -z6- -z3- -z4-', # wrong times for the 2Q gates
    'z0z5 -xz1- -xz2- -xz5- -xz4-', # wrong times for the 2Q gates
    'z0z5 -xz1- -xz2- -xz4- -xz4-', # wrong times for the 2Q gates
    'z3z0 -xx1- ----- -xz2- ----', # wrong meas time
]
rpng_simplified_examples = [
    '---- ---- ---- ----',
    '-z1- -z2- -z3- -z4-',
    '-z5- -x2- -x3- -z1-',
    '-x5h -z2z -x3x hz1-',
]
rpng_extended_examples = [
    'z0z6 ----- ----- ----- -----',
    'z0z3 -xz1- ----- -xz2- -----',
    'z0z5 -xz1- -xz2- -xz3- -xz4-',
]
for rpng in rpng_invalide_examples:
    try:
        RPNGDescription.from_string(corners_rpng_string = rpng)
    except ValueError as err:
        print(rpng)
        print(f'->  ERROR: {err}\n')
for rpng in rpng_simplified_examples:
    RPNGDescription.from_string(corners_rpng_string = rpng)
#for rpng in rpng_extended_examples:
#    RPNGDescription.from_string(corners_rpng_string = rpng)

----- ---- ---- ----
->  ERROR: The rpng string must be exactly 4-character long.

---- ---- --- ----
->  ERROR: The rpng string must be exactly 4-character long.

-z1- -z2- ---- -z4-
->  ERROR: Each plaquette must have 0, 2, or 4 2Q gates.

-z1- -z4- -z3- -z4-
->  ERROR: The n values for the corners must be unique.

-z1- -z6- -z3- -z4-
->  ERROR: The n values must be in ]0, ancilla meas time[.

z0z5 -xz1- -xz2- -xz5- -xz4-
->  ERROR: The rpng string must be exactly 4-character long.

z0z5 -xz1- -xz2- -xz4- -xz4-
->  ERROR: The rpng string must be exactly 4-character long.

z3z0 -xx1- ----- -xz2- ----
->  ERROR: The rpng string must be exactly 4-character long.



## 4. Example: associate a custom circuit to the plaquette

**TODO:** Here we just rephrased the method from `RPNGDescription`. Provide a different implementation or showcase a proper interface (still to be developed).

In [6]:
def custom_create_plaquette(desc: RPNGDescription, qubits: PlaquetteQubits = SquarePlaquetteQubits()) -> Plaquette:
    """Get the plaquette corresponding to the RPNG description
    
    Note that the ancilla qubit is the last among the PlaquetteQubits and thus
    has index 4, while the data qubits have indices 0-3.
    """
    prep_time = 0
    meas_time = desc.ancilla.n
    circuit_as_list = [''] * (meas_time - prep_time + 1)
    for q, rpng in enumerate(desc.corners):
        # 2Q gates.
        if rpng.n and rpng.p:
            if rpng.a.value == 'z':
                circuit_as_list[rpng.n] += f'C{rpng.p.value.upper()} 4 {q}\n'
            else:
                circuit_as_list[rpng.n] += f'C{rpng.a.value.upper()} {q} 4\n'
        # Data reset or Hadamard.
        if rpng.r:
            print(f'{q}: has reset -> {rpng.r}')
            circuit_as_list[0] += f'{rpng.get_r_op()} {q}\n'
        # Data measurement or Hadamard.
        if rpng.g:
            print(f'{q}: has meas -> {rpng.g}')
            circuit_as_list[-1] += f'{rpng.get_g_op()} {q}\n'
    # Ancilla reset and measurement.
    circuit_as_list[0] += f'R{desc.ancilla.r.value.upper()} 4\n'
    circuit_as_list[-1] += f'M{desc.ancilla.g.value.upper()} 4\n'
    q_map = QubitMap.from_qubits(qubits)
    circuit_as_str = "TICK\n".join(circuit_as_list)
    circuit = stim.Circuit(circuit_as_str)
    scheduled_circuit = ScheduledCircuit.from_circuit(circuit, qubit_map = q_map)
    return Plaquette(name = 'test', qubits = qubits, circuit = scheduled_circuit)

rpng_str = '-x5h -z2z -x3x hz1-'
qubits = SquarePlaquetteQubits()

try:
    desc = RPNGDescription.from_string(rpng_str)
    plaq = custom_create_plaquette(desc = desc, qubits = qubits)
    print(plaq.circuit.get_circuit(include_qubit_coords=True))
except ValueError as err:
    print(f'{err}')

0: has meas -> ExtendedBasisEnum.H
1: has meas -> ExtendedBasisEnum.Z
2: has meas -> ExtendedBasisEnum.X
3: has reset -> ExtendedBasisEnum.H
QUBIT_COORDS(-1, -1) 0
QUBIT_COORDS(1, -1) 1
QUBIT_COORDS(-1, 1) 2
QUBIT_COORDS(1, 1) 3
QUBIT_COORDS(0, 0) 4
H 3
RX 4
TICK
CZ 4 3
TICK
CZ 4 1
TICK
CX 4 2
TICK
TICK
CX 4 0
TICK
H 0
M 1
MX 2 4


----

## End